In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import torch
import torch.nn as nn
import csv
import os
import cv2
from PIL import Image
import matplotlib.pyplot as plt
import time

In [3]:
from prepare_data import *
from prepare_model import *

In [8]:
data = zigzag()

Successfully loaded NPZ.


In [9]:
data.print_data()

data path: ../1-generate/data/
start match: 0
end match: 0
start frame: 1
end frame: 120
available actions: [2, 3]
train data shape: (1, 120, 100, 100)
targets data shape: (1, 120, 1, 2)


In [12]:
data.train_data.shape

(1, 120, 100, 100)

In [13]:
X_train = data.train_data.reshape(1, 120, -1)
X_train.shape

(1, 120, 10000)

In [15]:
Y_train = data.targets.reshape(1, 120, 2)
Y_train.shape

(1, 120, 2)

In [16]:
X_train, Y_train = data.pack_to_model(X_train, Y_train)

In [17]:
X_train[0].shape

torch.Size([120, 10000])

In [18]:
data.data_shape

(1, 120, 100, 100)

In [19]:
data.target_shape

(1, 120, 1, 2)

In [ ]:
model_def = prepare_model(data)

1 - RNN
2 - LSTM
3 - CNN
Choose type of model: 2
LSTM model


In [14]:
model_def.get_info()

type: RNN
obs: teste_class
number of epochs: 100
number of hidden neurons: 200
using device: cpu
min losss: 1e-05
criterion MSELoss()
model path models/teste_class_m1to2_f1to1000_epoch100_H200


In [15]:
model = RNNModel(model_def.device, 
                 data.data_shape[2] * data.data_shape[3], 
                 data.target_shape[3], 
                 model_def.hidden_neurons, 
                 1, 
                 True)

In [21]:
model_def.train_def(model)

In [19]:
train_loss_arr = np.array([])
train_acc_arr = np.array([])

In [20]:
start_time_processing = time.time()

# Training Run
loss_file = open(model_def.model_path + '/' + "loss_file.txt", "w")
first_time = True

best_loss = 1
first_epoch = True

for epoch in range(1, model_def.n_epochs + 1):

    model.train()

    model_def.optimizer.zero_grad() # Clears existing gradients from previous epoch
    X_train.to(model_def.device)
    output = model(X_train)
    loss = model_def.criterion(output, Y_train.view(-1,len(data.actions_list)).float())
    loss.backward() # Does backpropagation and calculates gradients
    model_def.optimizer.step() # Updates the weights accordinglyw
        
    if epoch%10 == 0:

        train_loss_arr = np.append(train_loss_arr, loss.item())
        train_acc_arr  = np.append(train_acc_arr, model_def.get_acc(output, Y_train.reshape(-1, len(data.actions_list))))
        # train_acc_arr  = np.append(train_acc_arr, get_acc_2(output, target_padded.reshape(-1, len(ACTIONS_LIST))))
        
        loss_file.write("Epoch: {}/{}-------------------------------------------\n".format(epoch, model_def.n_epochs))
        loss_file.write("Train -> Loss: {:.15f} Acc: {:.15f}\n".format(train_loss_arr[-1], train_acc_arr[-1]))
            
        print("Epoch: {}/{}-------------------------------------------".format(epoch, model_def.n_epochs))
        print("Train -> Loss: {:.15f} Acc: {:.15f}".format(train_loss_arr[-1], train_acc_arr[-1]))
        
        if train_loss_arr[-1] < best_loss:
            state = { 'epoch': epoch + 1, 'state_dict': model.state_dict(),
                      'optimizer': model_def.optimizer.state_dict(), 'losslogger': loss.item(), }
            torch.save(state, model_def.model_path + '/' + model_def.model_name)
            best_loss = loss.item()
        else:
            print("model not saved")
            
loss_file.write("--- %s seconds ---" % (time.time() - start_time_processing))
loss_file.close()
np.savez(model_def.model_path + '/' + "train_loss_arr", train_loss_arr)
#np.savez(newpath + '/' + "valid_acc_table", valid_loss_mean_arr)
print("--- %s seconds ---" % (time.time() - start_time_processing))

C:\Users\alece\Desktop\UnB\tcc\rnn_enduro\pytorch_train\prepare_model.py:137: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  out = self.out(out)


Epoch: 10/100-------------------------------------------
Train -> Loss: 0.076540268957615 Acc: 0.481000006198883
Epoch: 20/100-------------------------------------------
Train -> Loss: 0.075322300195694 Acc: 0.481000006198883
Epoch: 30/100-------------------------------------------
Train -> Loss: 0.074391022324562 Acc: 0.481000006198883
Epoch: 40/100-------------------------------------------
Train -> Loss: 0.073414608836174 Acc: 0.481000006198883
Epoch: 50/100-------------------------------------------
Train -> Loss: 0.072337850928307 Acc: 0.481000006198883
Epoch: 60/100-------------------------------------------
Train -> Loss: 0.070846721529961 Acc: 0.567499995231628
Epoch: 70/100-------------------------------------------
Train -> Loss: 0.069291800260544 Acc: 0.572000026702881
Epoch: 80/100-------------------------------------------
Train -> Loss: 0.067875839769840 Acc: 0.589500010013580
Epoch: 90/100-------------------------------------------
Train -> Loss: 0.066359572112560 Acc: 0